# ADS 509 Module 1: APIs and Web Scraping

This notebook has two parts. In the first part, you will scrape lyrics from AZLyrics.com. In the second part, you'll run code that verifies the completeness of your data pull. 

For this assignment you have chosen two musical artists who have at least 20 songs with lyrics on AZLyrics.com. We start with pulling some information and analyzing them.


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


# Importing Libraries

In [25]:
import os
import datetime
import re

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random

In [26]:
# Use this cell for any import statements you add



---

# Lyrics Scrape

This section asks you to pull data by scraping www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways. 

In [27]:
artists = {'Taylor Swift':"https://www.azlyrics.com/t/taylorswift.html",
           'The Killers':"https://www.azlyrics.com/k/killers.html"} 
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).) 

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again. 

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages. 

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know? 

A: We are allowed to scrape on this page based on the rules stated in the robots.txt page. The user 008 cannot do this, however based on the * at the top portion all other users are allowed to scrape lyrics and songs. 


In [28]:
# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items() :
    # request the page and sleep
    r = requests.get(artist_page)
    time.sleep(5 + 10*random.random())

   # Check to see if the request was successful or not. 
    if r.status_code == 200:
        # Use beatifulsoup to parse through the HTML of each artist's page. 
        soup = BeautifulSoup(r.text, 'html.parser')

        # Now extract the links to lyrics pages from this page
        div_links = soup.find_all('div', class_='listalbum-item') 
        # store the links `lyrics_pages` where the key is the artist and the
        # value is a list of links. 
        links = [div.find('a')['href'] for div in div_links]
        lyrics_pages[artist] = links

    else:
        print(f"Failed to retrieve page for {artist}")

# Print links for each artist (Taylor Swift/The Killers)
for artist, links in lyrics_pages.items():
    print(f"Artist: {artist}")
    for link in links:
        print(f" - {link}")

Artist: Taylor Swift
 - /lyrics/taylorswift/samegirl.html
 - /lyrics/taylorswift/babyblue.html
 - /lyrics/taylorswift/neverfade.html
 - /lyrics/taylorswift/fire.html
 - /lyrics/taylorswift/maryjo.html
 - /lyrics/taylorswift/myturntobeme.html
 - /lyrics/taylorswift/kidinthecrowd.html
 - /lyrics/taylorswift/iusedtofly.html
 - /lyrics/taylorswift/luckyyou.html
 - /lyrics/taylorswift/pointofview.html
 - /lyrics/taylorswift/whydoyoutellme.html
 - /lyrics/taylorswift/houstonrodeo.html
 - /lyrics/taylorswift/alittlemorelikeyou.html
 - /lyrics/taylorswift/onesidedgoodbye.html
 - /lyrics/taylorswift/rideon.html
 - /lyrics/taylorswift/smokeyblacknights.html
 - /lyrics/taylorswift/halfwaytotexas.html
 - /lyrics/taylorswift/beautifuldays.html
 - /lyrics/taylorswift/yourheartssomewhereelse.html
 - /lyrics/taylorswift/timmcgraw.html
 - /lyrics/taylorswift/picturetoburn.html
 - /lyrics/taylorswift/teardropsonmyguitar.html
 - /lyrics/taylorswift/aplaceinthisworld.html
 - /lyrics/taylorswift/coldasyou.

Let's make sure we have enough lyrics pages to scrape. 

In [29]:
for artist, lp in lyrics_pages.items() :
    assert(len(set(lp)) > 20) 

In [30]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For Taylor Swift we have 412.
The full pull will take for this artist will take 1.14 hours.
For The Killers we have 139.
The full pull will take for this artist will take 0.39 hours.


## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part. 

1. Create an empty folder in our repo called "lyrics". 
1. Iterate over the artists in `lyrics_pages`. 
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages. 
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name. 


In [31]:
def generate_filename_from_link(link) :
    
    if not link :
        return None
    
    # drop the http or https and the html
    name = link.replace("https","").replace("http","")
    name = link.replace(".html","")

    name = name.replace("/lyrics/","")
    
    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    
    return name


In [32]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.
import shutil 

if os.path.isdir("lyrics") : 
    shutil.rmtree("lyrics/")

os.mkdir("lyrics")

In [40]:
from urllib.parse import urljoin

base_url = "https://www.azlyrics.com" 
start = time.time()

total_pages = 0 

def generate_filename_from_url(url):
    filename = ''.join(c if c.isalnum() else '_' for c in url)
    return filename + '.txt'

output_folder = 'lyrics'
song_limit = 20

for artist, links in lyrics_pages.items():
    artist_folder = os.path.join(output_folder, artist.lower())

    if not os.path.exists(artist_folder):
        os.makedirs(artist_folder)

    songs_scraped = 0

    for link in links:
        if songs_scraped >= song_limit:
            break
        
        absolute_url = urljoin(base_url, link)
        
        time.sleep(5 + 10 * random.random())

        
        response = requests.get(absolute_url)
        
        if response.status_code == 200:
            # Parse the HTML
            soup = BeautifulSoup(response.text, 'html.parser')
            

            title_element = soup.find('b')

            if title_element:
                
                title = title_element.get_text().strip()
        
                lyrics_text = ""

                br_tags = title_element.find_all_next('br')

                for br_tag in br_tags:
                    text = br_tag.find_next_sibling(text=True)
                    if text:
                        lyrics_text += text.strip() + "\n"

                lyrics_text = re.sub(r'<!--.*?-->', '', lyrics_text, flags=re.DOTALL)

                lyrics_text = lyrics_text.strip()

                if lyrics_text:
                    filename = generate_filename_from_url(link)
                    lyrics_file_path = os.path.join(artist_folder, filename)

                    with open(lyrics_file_path, 'w', encoding='utf-8') as lyrics_file:
                        lyrics_file.write(f"Title: {title}\n\n")
                        lyrics_file.write(f"Lyrics:\n{lyrics_text}")

                    print(f"Saved lyrics for {artist} - {filename}")

                    songs_scraped += 1

                    if songs_scraped >= song_limit:
                        print(f"Song limit reached for {artist}.")
                        break
                else:
                    print(f"No lyrics found for {artist} - {absolute_url}")
            else:
                print(f"No title found for {artist} - {absolute_url}")

end = time.time()
total_time = end - start


Saved lyrics for Taylor Swift - _lyrics_taylorswift_samegirl_html.txt
Saved lyrics for Taylor Swift - _lyrics_taylorswift_babyblue_html.txt
Saved lyrics for Taylor Swift - _lyrics_taylorswift_neverfade_html.txt
Saved lyrics for Taylor Swift - _lyrics_taylorswift_fire_html.txt
Saved lyrics for Taylor Swift - _lyrics_taylorswift_maryjo_html.txt
Saved lyrics for Taylor Swift - _lyrics_taylorswift_myturntobeme_html.txt
Saved lyrics for Taylor Swift - _lyrics_taylorswift_kidinthecrowd_html.txt
Saved lyrics for Taylor Swift - _lyrics_taylorswift_iusedtofly_html.txt
Saved lyrics for Taylor Swift - _lyrics_taylorswift_luckyyou_html.txt
Saved lyrics for Taylor Swift - _lyrics_taylorswift_pointofview_html.txt
Saved lyrics for Taylor Swift - _lyrics_taylorswift_whydoyoutellme_html.txt
Saved lyrics for Taylor Swift - _lyrics_taylorswift_houstonrodeo_html.txt
Saved lyrics for Taylor Swift - _lyrics_taylorswift_alittlemorelikeyou_html.txt
Saved lyrics for Taylor Swift - _lyrics_taylorswift_onesidedg

In [41]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

Total run time was 0.15 hours.


---

# Evaluation

This assignment asks you to pull data by scraping www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [42]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

## Checking Lyrics 

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [43]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")


For the killers we have 20 files.
For the killers we have roughly 5147 words, 758 are unique.
For taylor swift we have 20 files.
For taylor swift we have roughly 5619 words, 820 are unique.
